# LangGraph ReAct Agent Example

In this cookbook, we'll demonstrate a basic LangGraph ReAct agent using Exfunc.

## Step 1: Set up the environment

First, let's install the required libraries and set up the SDK and clients.

In [ ]:
%pip install -U langgraph langchain langchain-openai

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API key:\n")
if "EXFUNC_API_KEY" not in os.environ:
    os.environ["EXFUNC_API_KEY"] = getpass.getpass("Exfunc API key:\n")

In [ ]:
from langchain_openai import ChatOpenAI
from exfunc import Exfunc

model = ChatOpenAI(model="gpt-4o-mini")
exfunc = Exfunc(api_key=os.getenv("EXFUNC_API_KEY"))

## Step 2: Add Exfunc API as custom tools

Next, we will add Exfunc API as custom tools. In this example, we added Yelp SearchBusinesses and GetBusinessReviews API.

In [ ]:
from langchain_core.tools import tool

@tool
def search_businesses(query: str, location: str) -> list:
    """Search businesses on Yelp."""
    search_businesses_result = exfunc.yelp.search_businesses(request={
        "query": query,
        "location": location,
    })
    return search_businesses_result.businesses

@tool
def get_business_reviews(business_id: str) -> dict:
    """Get a list of reviews on Yelp for a given business ID."""
    get_business_reviews_result = exfunc.yelp.get_business_reviews(request={
        "business_id": business_id,
    })
    return get_business_reviews_result.reviews


tools = [search_businesses, get_business_reviews]

## Step 3: Create a pre-built ReAct agent with custom tools

You can easily create a ReAct agent with custom tools using the create_react_agent helper function.

In [ ]:
from langgraph.prebuilt import create_react_agent

langgraph_agent_executor = create_react_agent(model, tools)

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [ ]:
query = "analyze reviews of yin du wonton noodle soup in SF"
inputs = {"messages": [("user", query)]}
message = print_stream(langgraph_agent_executor.stream(inputs, stream_mode="values"))

## Conclusion

That's it! By following this cookbook, you can easily create a LangGraph ReAct agent using Exfunc.

If you have any questions or need help, feel free to reach out to us at [Exfunc](https://exfunc.com/).